In [16]:
# %%python3 -m ipykernel install --user --name=musiclm

%pip install musiclm-pytorch torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 2.8 MB/s eta 0:00:00:00:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 27.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 35.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2
  Attempting uninstall: torchdata
    Found exist

In [2]:
import torch
from musiclm_pytorch import MuLaN, AudioSpectrogramTransformer, TextTransformer

audio_transformer = AudioSpectrogramTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64,
    spec_n_fft = 128,
    spec_win_length = 24,
    spec_aug_stretch_factor = 0.8
)

text_transformer = TextTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64,
)

mulan = MuLaN(
    audio_transformer = audio_transformer,
    text_transformer = text_transformer,
)

wavs = torch.randn(2,1024)
texts = torch.randint(0, 20000, (2, 256))

loss = mulan(wavs, texts)
loss.backward()

embeds=mulan.get_audio_latents(wavs) # during training
# embeds=mulan.get_audio_latents(texts) # during inference

spectrogram yielded shape of (65, 86), but had to be cropped to (64, 80) to be patchified for transformer


In [4]:
from musiclm_pytorch import MuLaNEmbedQuantizer

quantizer = MuLaNEmbedQuantizer(
    mulan = mulan,
    conditioning_dims = (1024,1024,1024),
    namespaces = ('semantics', 'coarse', 'fine'),
)

# now say you want the conditioning embeddings for semantic transformer

wavs = torch.randn(2,1024)
conds = quantizer(wavs=wavs, namespace='semantics') # (2,8,1024) - 8 is number of quantizers

In [5]:
import torch
from audiolm_pytorch import HubertWithKmeans, SemanticTransformer, SemanticTransformerTrainer

wav2vec = HubertWithKmeans(
    checkpoint_path = './hubert/hubert_base_ls960.pt',
    kmeans_path = './hubert/hubert_base_ls960_L9_km500.bin'
)

semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6,
    audio_text_condition = True      # this must be set to True (same for CoarseTransformer and FineTransformers)
).cuda()

trainer = SemanticTransformerTrainer(
    transformer = semantic_transformer,
    wav2vec = wav2vec,
    audio_conditioner = quantizer,   # pass in the MulanEmbedQuantizer instance above
    folder ='/path/to/audio/files',
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 1
)

trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:16                                                                                   │
│                                                                                                  │
│   13 │   audio_text_condition = True      # this must be set to True (same for CoarseTransfor    │
│   14 ).cuda()                                                                                    │
│   15                                                                                             │
│ ❱ 16 trainer = SemanticTransformerTrainer(                                                       │
│   17 │   transformer = semantic_transformer,                                                     │
│   18 │   wav2vec = wav2vec,                                                                      │
│   19 │   audio_conditioner = quantizer,   # pass in the MulanEmbedQuantizer instance above       │
│ in __init__:116                                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/audiolm_pytorch/trainer.py:606 in __init__               │
│                                                                                                  │
│    603 │   │   │   if exists(data_max_length_seconds):                                           │
│    604 │   │   │   │   data_max_length = data_max_length_seconds * wav2vec.target_sample_hz      │
│    605 │   │   │                                                                                 │
│ ❱  606 │   │   │   self.ds = SoundDataset(                                                       │
│    607 │   │   │   │   folder,                                                                   │
│    608 │   │   │   │   max_length = data_max_length,                                             │
│    609 │   │   │   │   target_sample_hz = wav2vec.target_sample_hz,                              │
│ in __init__:116                                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/audiolm_pytorch/data.py:46 in __init__                   │
│                                                                                                  │
│    43 │   ):                                                                                     │
│    44 │   │   super().__init__()                                                                 │
│    45 │   │   path = Path(folder)                                                                │
│ ❱  46 │   │   assert path.exists(), 'folder does not exist'                                      │
│    47 │   │                                                                                      │
│    48 │   │   files = [file for ext in exts for file in path.glob(f'**/*.{ext}')]                │
│    49 │   │   assert len(files) > 0, 'no sound files found'                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: folder does not exist